In [5]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.autograd import Variable
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import time

learning_rate = 1e-3
batch_size = 64
epoches = 50

trans_img = transforms.ToTensor()

trainset = MNIST('./data', train=True, transform=trans_img)
testset = MNIST('./data', train=False, transform=trans_img)

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

# build network
class Lenet(nn.Module):
    def __init__(self):
        super(Lenet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, 3, stride=1, padding=1),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5, stride=1, padding=0),
            nn.MaxPool2d(2, 2)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(400, 120),
            nn.Linear(120, 84),
            nn.Linear(84, 10)
        )
        
        
    def forward(self, x):
        out = self.conv(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

lenet = Lenet()
lenet.cuda()

criterian = nn.CrossEntropyLoss(size_average=False)
optimizer = optim.SGD(lenet.parameters(), lr=learning_rate)

# train
for i in range(epoches):
    since = time.time()
    running_loss = 0.
    running_acc = 0.
    for (img, label) in trainloader:
        img = Variable(img).cuda()
        label = Variable(label).cuda()
        
        optimizer.zero_grad()
        output = lenet(img)
        loss = criterian(output, label)
        # backward
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data[0]
        _, predict = torch.max(output, 1)
        correct_num = (predict == label).sum()
        running_acc += correct_num.data[0]
    
    running_loss /= len(trainset)
    running_acc /= len(trainset)
    print("[%d/%d] Loss: %.5f, Acc: %.2f, Time: %.1f s" %(i+1, epoches, running_loss, 100*running_acc, time.time()-since))

[1/50] Loss: 0.42389, Acc: 86.35, Time: 3.4 s
[2/50] Loss: 0.10881, Acc: 96.67, Time: 3.7 s
[3/50] Loss: 0.08289, Acc: 97.48, Time: 3.5 s
[4/50] Loss: 0.06887, Acc: 97.89, Time: 3.7 s
[5/50] Loss: 0.06077, Acc: 98.09, Time: 3.5 s
[6/50] Loss: 0.05484, Acc: 98.31, Time: 3.8 s
[7/50] Loss: 0.04954, Acc: 98.45, Time: 3.6 s
[8/50] Loss: 0.04534, Acc: 98.57, Time: 3.7 s
[9/50] Loss: 0.04253, Acc: 98.67, Time: 3.6 s
[10/50] Loss: 0.04110, Acc: 98.71, Time: 3.7 s
[11/50] Loss: 0.03782, Acc: 98.77, Time: 3.6 s
[12/50] Loss: 0.03587, Acc: 98.87, Time: 3.6 s
[13/50] Loss: 0.03356, Acc: 98.88, Time: 3.6 s
[14/50] Loss: 0.03267, Acc: 98.94, Time: 3.6 s
[15/50] Loss: 0.03037, Acc: 99.04, Time: 3.6 s
[16/50] Loss: 0.02808, Acc: 99.11, Time: 3.5 s
[17/50] Loss: 0.02749, Acc: 99.14, Time: 3.4 s
[18/50] Loss: 0.02691, Acc: 99.10, Time: 3.6 s
[19/50] Loss: 0.02525, Acc: 99.16, Time: 3.5 s
[20/50] Loss: 0.02455, Acc: 99.20, Time: 3.5 s
[21/50] Loss: 0.02329, Acc: 99.23, Time: 3.5 s
[22/50] Loss: 0.02152,

In [6]:
# evaluate
lenet.eval()

testloss = 0.
testacc = 0.
for (img, label) in testloader:
    img = Variable(img).cuda()
    label = Variable(label).cuda()
    
    output = lenet(img)
    loss = criterian(output, label)
    testloss += loss.data[0]
    _, predict = torch.max(output, 1)
    num_correct = (predict == label).sum()
    testacc += num_correct.data[0]

testloss /= len(testset)
testacc /= len(testset)
print("Test: Loss: %.5f, Acc: %.2f %%" %(testloss, 100*testacc))

Test: Loss: 0.07444, Acc: 98.46 %
